<a href="https://www.kaggle.com/code/aritrapal3196/nlp-3rd-yr-6th-project?scriptVersionId=97440760" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/private-dataset1/NLP_final_with_tags.csv


In [2]:
#Made by Aritra Pal,Archisman Roy Chowdhury,Rahul Yadav
#Students from Institute of Enginnering and Management,Kolkata
#Topic - Twitter Sentiment Analysis

In [3]:
#Importing necessary libraries
import re
import numpy as np
import pandas as pd

import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer

from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
#Reading the dataset
item = pd.read_csv('../input/private-dataset1/NLP_final_with_tags.csv')
df = pd.DataFrame(item)
df.head()

,Tweets,@ Name,Support,War
0,@marceldirsus And even if the war was over in ...,szpaked,Ukraine,No
1,"""The Russian military may feel it is just warm...",Mike_Eckel,Neutral,Neutral
2,"@JuliaDavisNews Let's see, Russia attacks Ukra...",Purpmongoose,Ukraine,Neutral
3,@RepAnnWagner You are able to change the #Ukra...,icosagon20,Ukraine,Yes
4,So let's get this straight:\nBecause Ukraine s...,PaladinoEU,Ukraine,Neutral


In [5]:
df['Tweets']=df['Tweets'].str.lower()
df.head()

,Tweets,@ Name,Support,War
0,@marceldirsus and even if the war was over in ...,szpaked,Ukraine,No
1,"""the russian military may feel it is just warm...",Mike_Eckel,Neutral,Neutral
2,"@juliadavisnews let's see, russia attacks ukra...",Purpmongoose,Ukraine,Neutral
3,@repannwagner you are able to change the #ukra...,icosagon20,Ukraine,Yes
4,so let's get this straight:\nbecause ukraine s...,PaladinoEU,Ukraine,Neutral


In [6]:
#list of stopwords
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will',  'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [7]:
#Stopwords removal
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(Tweets):
    return " ".join([word for word in str(Tweets).split() if word not in STOPWORDS])
df['Tweets'] = df['Tweets'].apply(lambda Tweets: cleaning_stopwords(Tweets))
df['Tweets'].head()

0    @marceldirsus even war over 2 months, still ne...
1    "the russian military may feel warming up. syr...
2    @juliadavisnews let's see, russia attacks ukra...
3    @repannwagner able change #ukraine-russia war ...
4    let's get straight: ukraine sank moskva (which...
Name: Tweets, dtype: object

In [8]:
#Punctuations removal
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(Tweets):
    translator = str.maketrans('', '', punctuations_list)
    return Tweets.translate(translator)
df['Tweets']= df['Tweets'].apply(lambda x: cleaning_punctuations(x))
df['Tweets'].head()

0    marceldirsus even war over 2 months still need...
1    the russian military may feel warming up syria...
2    juliadavisnews lets see russia attacks ukraine...
3    repannwagner able change ukrainerussia war thi...
4    lets get straight ukraine sank moskva which ru...
Name: Tweets, dtype: object

In [9]:
#Repeating character removal
def cleaning_repeating_char(Tweets):
    return re.sub(r'(.)1+', r'1', Tweets)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_repeating_char(x))
df['Tweets'].head()

0    marceldirsus even war over 2 months still need...
1    the russian military may feel warming up syria...
2    juliadavisnews lets see russia attacks ukraine...
3    repannwagner able change ukrainerussia war thi...
4    lets get straight ukraine sank moskva which ru...
Name: Tweets, dtype: object

In [10]:
#URL and Hyperlink removal
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_URLs(x))
df['Tweets'].head()

0    marceldirsus even war over 2 months still need...
1    the russian military may feel warming up syria...
2    juliadavisnews lets see russia attacks ukraine...
3    repannwagner able change ukrainerussia war thi...
4    lets get straight ukraine sank moskva which ru...
Name: Tweets, dtype: object

In [11]:
#Number removal
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_numbers(x))
df['Tweets'].head()

0    marceldirsus even war over  months still need ...
1    the russian military may feel warming up syria...
2    juliadavisnews lets see russia attacks ukraine...
3    repannwagner able change ukrainerussia war thi...
4    lets get straight ukraine sank moskva which ru...
Name: Tweets, dtype: object

In [12]:
#importing nltkn and downloading vader_lexicon for sentiment analysis
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [14]:
sid = SentimentIntensityAnalyzer()

In [15]:
#generating polarities
df['polarity scores'] = df['Tweets'].apply(lambda x: sid.polarity_scores(x))

In [16]:
df.head()

,Tweets,@ Name,Support,War,polarity scores
0,marceldirsus even war over months still need ...,szpaked,Ukraine,No,"{'neg': 0.298, 'neu': 0.702, 'pos': 0.0, 'comp..."
1,the russian military may feel warming up syria...,Mike_Eckel,Neutral,Neutral,"{'neg': 0.2, 'neu': 0.718, 'pos': 0.082, 'comp..."
2,juliadavisnews lets see russia attacks ukraine...,Purpmongoose,Ukraine,Neutral,"{'neg': 0.377, 'neu': 0.527, 'pos': 0.096, 'co..."
3,repannwagner able change ukrainerussia war thi...,icosagon20,Ukraine,Yes,"{'neg': 0.192, 'neu': 0.44, 'pos': 0.368, 'com..."
4,lets get straight ukraine sank moskva which ru...,PaladinoEU,Ukraine,Neutral,"{'neg': 0.404, 'neu': 0.485, 'pos': 0.111, 'co..."


In [17]:
df['polarity'] = df['polarity scores'].apply(lambda x: x['compound'])
print("The polarity scores and overall polarity of the dataset(first 10) is shown:")
df.head(10)
# Thus we see that our dataset is well balanced and not biased towards any party 
# and contains tweets of all polarities

The polarity scores and overall polarity of the dataset(first 10) is shown:


,Tweets,@ Name,Support,War,polarity scores,polarity
0,marceldirsus even war over months still need ...,szpaked,Ukraine,No,"{'neg': 0.298, 'neu': 0.702, 'pos': 0.0, 'comp...",-0.8807
1,the russian military may feel warming up syria...,Mike_Eckel,Neutral,Neutral,"{'neg': 0.2, 'neu': 0.718, 'pos': 0.082, 'comp...",-0.5106
2,juliadavisnews lets see russia attacks ukraine...,Purpmongoose,Ukraine,Neutral,"{'neg': 0.377, 'neu': 0.527, 'pos': 0.096, 'co...",-0.8898
3,repannwagner able change ukrainerussia war thi...,icosagon20,Ukraine,Yes,"{'neg': 0.192, 'neu': 0.44, 'pos': 0.368, 'com...",0.6597
4,lets get straight ukraine sank moskva which ru...,PaladinoEU,Ukraine,Neutral,"{'neg': 0.404, 'neu': 0.485, 'pos': 0.111, 'co...",-0.9195
5,world leaders say war ukraine strengthens argu...,sahagunfelipe,Neutral,Neutral,"{'neg': 0.27, 'neu': 0.59, 'pos': 0.14, 'compo...",-0.5131
6,every war crime russia committing ukraine alre...,Quzulkurt_edi,Neutral,Neutral,"{'neg': 0.303, 'neu': 0.602, 'pos': 0.096, 'co...",-0.7814
7,coribush stephcongress able change ukraineruss...,icosagon20,Ukraine,Yes,"{'neg': 0.192, 'neu': 0.44, 'pos': 0.368, 'com...",0.6597
8,pontifex pontifex ukraine russia its not confl...,SGafijka,Ukraine,No,"{'neg': 0.541, 'neu': 0.418, 'pos': 0.041, 'co...",-0.9748
9,benjaminlasnier russia obviously lost sence re...,OleksandraTat2,Ukraine,Yes,"{'neg': 0.374, 'neu': 0.408, 'pos': 0.218, 'co...",-0.8176


In [18]:
print("No. of tweets which want WAR:")
print(len(df.loc[df['War'] == "No"]))

No. of tweets which want WAR:
499


In [19]:
print("No. of tweets which do not want WAR:")
print(len(df.loc[df['War'] == "Yes"]))

No. of tweets which do not want WAR:
506


In [20]:
print("No. of tweets which support Russia:")
print(len(df.loc[df['Support'] == "Russia"]))

No. of tweets which support Russia:
490


In [21]:
print("No. of tweets which support Ukraine:")
print(len(df.loc[df['Support'] == "Ukraine"]))

No. of tweets which support Ukraine:
468


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
 
X = cv.fit_transform(df.Tweets).toarray()
y = df.iloc[:, 2].values

In [23]:
df.dropna(inplace=True)
from sklearn.model_selection import train_test_split
#Splitting dataset into training(80%) and testing(20%) dataset
train,test = train_test_split(df, test_size = 0.2, random_state = 42)

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
train.Tweets.dropna()
train.Support.dropna()
text_clf.fit(train.Tweets, train.Support)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [25]:
#MultinomialNB for Support
predicted = text_clf.predict(test.Tweets)
np.mean(predicted == test.Support)

0.3612040133779264

In [26]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
train.Tweets.dropna()
train.Support.dropna()
text_clf.fit(train.Tweets, train.Support)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [27]:
#SGDClassifier for Support
predicted = text_clf.predict(test.Tweets)
np.mean(predicted == test.Support)

0.35785953177257523

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
train.Tweets.dropna()
train.War.dropna()
text_clf.fit(train.Tweets, train.War)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [29]:
#MultinomialNB for War
predicted = text_clf.predict(test.Tweets)
np.mean(predicted == test.War)

0.3745819397993311

In [30]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
train.Tweets.dropna()
train.War.dropna()
text_clf.fit(train.Tweets, train.War)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [31]:
#SGDClassifier for War
predicted = text_clf.predict(test.Tweets)
np.mean(predicted == test.War)

0.3612040133779264

In [32]:
#confusion matrix
from sklearn.metrics import classification_report, confusion_matrix


print(confusion_matrix(test.Support, predicted))

[[43 33  0  0 34]
 [ 0  0  0  0  0]
 [35 35  0  0 32]
 [32 25  0  0 30]
 [ 0  0  0  0  0]]


In [33]:
#classification report
from sklearn import metrics
target_names = ['Russia','Ukraine','Neutral','Yes','No']

print(metrics.classification_report(test.Support, predicted,  target_names=target_names))

              precision    recall  f1-score   support

      Russia       0.39      0.39      0.39       110
     Ukraine       0.00      0.00      0.00         0
     Neutral       0.00      0.00      0.00       102
         Yes       0.00      0.00      0.00        87
          No       0.00      0.00      0.00         0

    accuracy                           0.14       299
   macro avg       0.08      0.08      0.08       299
weighted avg       0.14      0.14      0.14       299



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1

In [34]:
# We used MultinomialNB and SGDClassifier as both of them are classification algorithms and are used for high-dimensional training dataset

In [35]:
#Conclusion:
# For Support for Russia-Ukraine, SGDClassifier gives better accuracy
# For Support for War, MultinomialNB gives better accuracy